<a href="https://colab.research.google.com/github/akashdhanpal/Pnemonia_Detection_CV_Deep_Learning/blob/main/BASIC_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount("/content/gdrive", force_remount=True)

In [ ]:
pwd

'C:\\Users\\ajith'

In [ ]:
# General libraries
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf


# Setting seeds for reproducibility
seed = 232
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
input_path = 'rsna-pneumonia-detection-challenge/'

In [ ]:
ls

 Volume in drive C is AJITH
 Volume Serial Number is 8ABB-AD60

 Directory of C:\Users\ajith

21-11-2020  12:40    <DIR>          .
21-11-2020  12:40    <DIR>          ..
20-12-2019  21:11    <DIR>          .anaconda
07-06-2020  16:03    <DIR>          .conda
02-01-2020  20:54                43 .condarc
21-11-2020  10:00    <DIR>          .config
07-11-2020  14:36    <DIR>          .ipynb_checkpoints
25-12-2019  18:58    <DIR>          .ipython
19-07-2020  13:56    <DIR>          .jupyter
11-10-2020  16:05    <DIR>          .keras
02-01-2020  21:01    <DIR>          .matplotlib
23-01-2020  20:19    <DIR>          .Origin
20-12-2019  21:15    <DIR>          .PyCharmCE2019.3
07-11-2020  13:41               244 .python_history
23-01-2020  20:19    <DIR>          .QtWebEngineProcess
07-06-2020  16:03    <DIR>          .surprise_data
19-07-2020  13:55             8,058 0.0.7
17-11-2020  21:32    <DIR>          3D Objects
05-11-2020  19:59    <DIR>          Apple
02-01-2020  20:59           

In [ ]:
from os import path 
input_path+'stage_2_train_images'
path.isfile(input_path+'stage_2_train_images/'+'2c781913-5f24-4b0e-959c-6b7385c6fce0.dcm')


True

In [ ]:
def process_data(img_dims, batch_size):
    # Data generation objects
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    # This is fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'train', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(
    directory=input_path+'test-1', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='binary', 
    shuffle=True)
    
    return train_gen, test_gen

In [ ]:

# Hyperparameters
img_dims = 150
epochs = 10
batch_size = 32

# Getting the data
train_gen, test_gen = process_data(img_dims, batch_size)

Found 25803 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:

# Input layer
inputs = Input(shape=(img_dims, img_dims, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=1, activation='sigmoid')(x)


In [ ]:
# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint(filepath='best_weights.hdf5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=2, verbose=2, mode='max')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [ ]:
# Fitting the model
hist = model.fit_generator(
           train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=epochs, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

Epoch 1/10
806/806 [==============================] - 899s 1s/step - loss: 0.4659 - accuracy: 0.7893 - val_loss: 0.5959 - val_accuracy: 0.7533
Epoch 2/10
806/806 [==============================] - 729s 905ms/step - loss: 0.4532 - accuracy: 0.7953 - val_loss: 0.5573 - val_accuracy: 0.5321
Epoch 3/10
806/806 [==============================] - 710s 881ms/step - loss: 0.4402 - accuracy: 0.7980 - val_loss: 0.7665 - val_accuracy: 0.3851
Epoch 4/10
806/806 [==============================] - 641s 795ms/step - loss: 0.4332 - accuracy: 0.8027 - val_loss: 0.6992 - val_accuracy: 0.3801
Epoch 5/10
806/806 [==============================] - 505s 627ms/step - loss: 0.4313 - accuracy: 0.8005 - val_loss: 0.6667 - val_accuracy: 0.4662

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 6/10
806/806 [==============================] - 502s 622ms/step - loss: 0.4095 - accuracy: 0.8123 - val_loss: 0.6284 - val_accuracy: 0.5186
Epoch 7/10
806/806 [==========================

In [ ]:

    test_data = []
    test_labels = []

    for cond in ['/NORMAL/', '/PNEUMONIA/']:
        for img in (os.listdir(input_path + 'test-1' + cond)):
            img = plt.imread(input_path+'test-1'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/NORMAL/':
                label = 0
            elif cond=='/PNEUMONIA/':
                label = 1
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

preds = model.predict(test_data)

acc = accuracy_score(test_labels, np.round(preds))*100
cm = confusion_matrix(test_labels, np.round(preds))
tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')
print(cm)

print('\nTEST METRICS ----------------------')
precision = tp/(tp+fp)*100
recall = tp/(tp+fn)*100
print('Accuracy: {}%'.format(acc))
print('Precision: {}%'.format(precision))
print('Recall: {}%'.format(recall))
print('F1-score: {}'.format(2*precision*recall/(precision+recall)))

print('\nTRAIN METRIC ----------------------')
print('Train acc: {}'.format(np.round((hist.history['accuracy'][-1])*100, 2)))

CONFUSION MATRIX ------------------
[[230   4]
 [251 139]]

TEST METRICS ----------------------
Accuracy: 59.13461538461539%
Precision: 97.2027972027972%
Recall: 35.64102564102564%
F1-score: 52.15759849906192

TRAIN METRIC ----------------------
Train acc: 81.74


In [ ]:
from keras.preprocessing import image


In [ ]:
img = plt.imread('chest_xray/val/PNEUMONIA/person1954_bacteria_4886.jpeg')
img = cv2.resize(img, (img_dims, img_dims))
img = np.dstack([img, img, img])
img = img.astype('float32') / 255
result = model.predict(np.expand_dims(image.img_to_array(img), axis = 0))  

if result[0][0] > 0.5:
  prediction = 'Pnuemonia'
else:
  prediction = 'Normal'
  
print(prediction)

In [ ]:
pwd

'C:\\Users\\ajith'

IMAGE CONVERSION FROM DICOM TO **JPG**

In [ ]:
import pydicom as dicom
import os
import cv2
#import PIL # optional
# make it True if you want in PNG format
PNG = False
# Specify the .dcm folder path

folder_path = "rsna-pneumonia-detection-challenge/stage_2_test_images"
# Specify the output jpg/png folder path
jpg_folder_path = "rsna-pneumonia-detection-challenge/test"
images_path = os.listdir(folder_path)
for n, image in enumerate(images_path):
    ds = dicom.dcmread(os.path.join(folder_path, image))
    pixel_array_numpy = ds.pixel_array
    if PNG == False:
        image = image.replace('.dcm', '.jpg')
    else:
        image = image.replace('.dcm', '.png')
    cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)
    if n % 50 == 0:
        print('{} image converted'.format(n))

In [ ]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 75, 75, 32)        688       
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 75, 75, 32)        1344      
_________________________________________________________________
batch_normalization_1 (Batch (None, 75, 75, 32)        128 